# SameSentiment Yelp - Category Evaluation

In [ ]:
import json
import pickle
from pathlib import Path

from tqdm import tqdm
from transformers.trainer_utils import set_seed

tqdm.pandas()

In [ ]:
# download + scp to server + extract
data_yelp_path = Path("data/sentiment/yelp/")

# ------------------------------------

# local?
data_yelp_path = Path("data_raw/sentiment/yelp/")

# local? - output path (base) for sentiment review yelp pairs
data_yelp_b_tdt_path = Path("data/sentiment/yelp-pair-b/")
data_yelp_b_rand_tdt_path = Path("data/sentiment/yelp-pair-rand-b/")
# local - output path for simple sentiment reviews yelp
data_yelp_tdt_sentiment_5_path = Path("data/sentiment/yelp-sentiment-5/")
data_yelp_tdt_sentiment_b_path = Path("data/sentiment/yelp-sentiment-b/")

---

In [3]:
dn_yelp_cached = data_yelp_path / "cached"

In [4]:
#  #### Load categories & topics
from data_prep import load_reviews, load_topics

# ##### Filter categories
from data_prep import filter_min_cat_combis, make_map_cats, make_cat_combis

# ##### Filter reviews
from data_prep import filter_min_review_freq, filter_both_good_bad


# #### Load category tree
from data_prep import load_category_tree
from data_prep import get_root_category_items

# #### Filter other category businesses
from data_prep import filter_root_category_businesses_not_other


# #### Cache root category reviews in dataframes
from data_prep import cache_root_category_businesses_df


# #### Dataframe for training etc.
from data_prep import make_or_load_pairs


# #### Make train/dev/test splits
from data_prep import write_pair_df_tsv


---

## Run

#### Load reviews

In [5]:
fn_yelp_reviews = data_yelp_path / "review.json"
df = load_reviews(fn_yelp_reviews)

6685900it [00:38, 175531.62it/s]


#### Load categories for businesses

- business (id) with list of topics/categories
- lookups (business -> categories, category -> businesses)
- list of combinations (with amount)

In [ ]:
fn_yelp_topics = data_yelp_path / "business.json"
bids_not_cats = set()
inv_bid_cats = load_topics(fn_yelp_topics, bids_not_cats=bids_not_cats)

inv_cat_bids = make_map_cats(inv_bid_cats)

inv_cat_combis = make_cat_combis(inv_bid_cats)

#### Load category tree

- hierarchy of categories

In [7]:
fn_yelp_catgory_tree = data_yelp_path / "all_category_list.json"
map_categories, map_cat_name2id, lst_root_categories = load_category_tree(fn_yelp_catgory_tree)

#### Pre-Cache all root category businesses (reviews)

In [8]:
cache_root_category_businesses_df(df, inv_cat_bids, map_categories, map_cat_name2id)

---

## Run test evaluation per category

In [9]:
model_name = "bert-base-uncased"
#model_name = "bert-base-cased"
#model_name = "distilroberta-base"
#model_name = "albert-base-v2"

data_name = "yelp-pair-b"
#data_name = "yelp-pair-rand-b"

seq_len = 256
batch_size = 16
acc_steps = 64
num_epoch = 3
cuda_devs = "1"

category_name = "*"

run_name = f"{model_name}-{data_name}_{seq_len}_{batch_size}_{num_epoch}"
run_name = f"{model_name.replace('/', '-')}-{data_name}_{seq_len}_{batch_size}-acc{acc_steps}_{num_epoch}"

fn_cateval_base = Path(f"./output_cateval/{run_name}")
fn_cateval_base.mkdir(parents=True, exist_ok=True)

In [10]:
# lst_root_categories
root_categories = get_root_category_items(map_categories)
root_categories = sorted(root_categories, key=lambda x: x["title"])
root_category_labels = [x["title"] for x in root_categories]

map_root_category_labels2alias = {x["title"]: x["alias"] for x in root_categories}
map_root_category_alias2label = {x["alias"]: x["title"] for x in root_categories}

In [ ]:
for root_category_label in root_category_labels:
    if root_category_label == category_name:
        continue
    print(f"Work on {root_category_label} ...")

    # prepare test data, without overlap
    df_other = filter_root_category_businesses_not_other(dn_yelp_cached, root_category_label, category_name, inv_cat_bids, map_categories, map_cat_name2id)
    if df_other.shape[0] == 0:
        continue

    df_other = filter_min_review_freq(df_other, min_ratings=10)
    df_other = filter_both_good_bad(df_other)
    if df_other.shape[0] == 0:
        continue

    df_other = make_or_load_pairs(df_other, inv_cat_bids, None, num_pairs_per_class=4)
    print(f"review pairs: {len(df_other)}")

    # check paths for writing
    root_category_alias = map_root_category_labels2alias[root_category_label]
    fn_data_path = Path(f"data/sentiment/{run_name}/{root_category_alias}")
    fn_data_path.mkdir(parents=True, exist_ok=True)

    # write test tsv
    write_pair_df_tsv(df_other, fn_data_path / "test.tsv", "test")
    
    # evaluate
    ! CUDA_VISIBLE_DEVICES={cuda_devs} \
        python trainer.py \
        --do_test \
        --model_name_or_path ./output_sent/{run_name} \
        --task_name same-b \
        --data_dir {fn_data_path} \
        --output_dir ./output_cateval/{run_name}/category/{root_category_alias} \
        --run_name {run_name}-{root_category_alias} \
        --per_device_eval_batch_size {batch_size} \
        --max_seq_length {seq_len} \
        --overwrite_cache \

    print()

In [24]:
fn_base = fn_cateval_base / "category"

# search for runs
data = list()
for fn in sorted(fn_base.iterdir()):
    cat_name = fn.name

    sfn = fn / "test_results_same-b.json"
    if not sfn.exists():
        print(f"No result: {cat_name}")
        continue

    with (fn / "test_results_same-b.json").open("r") as fp:
        stats_test = json.load(fp)
        
    data.append((cat_name, stats_test))

# sort by test accuracy descending
data = sorted(data, key=lambda x: x[1]["eval_acc"], reverse=True)

# output
print()
print(f"{run_name} results:")
print(f"  CATEGORY----------------------  ACC-- (F1---)")
for cat_name, stats_test in data:
    print(f"  {cat_name:<30.30}  {stats_test['eval_acc'] * 100:.2f} ({stats_test['eval_f1'] * 100:.2f})")
    #   ({stats_test['eval_class_report']['macro avg']['precision'] * 100:.2f}  {stats_test['eval_class_report']['macro avg']['recall'] * 100:.2f})


bert-base-uncased-yelp-pair-b_256_16-acc64_3 results:
  CATEGORY----------------------  ACC-- (F1---)
  health                          92.66 (92.68)
  beautysvc                       92.56 (92.57)
  homeservices                    92.22 (92.28)
  financialservices               91.92 (92.02)
  auto                            91.84 (91.86)
  pets                            91.20 (91.23)
  localservices                   90.99 (91.01)
  professional                    90.89 (90.97)
  massmedia                       86.93 (86.55)
  shopping                        86.70 (86.77)
  education                       86.35 (86.41)
  religiousorgs                   85.16 (85.38)
  eventservices                   84.63 (84.76)
  hotelstravel                    84.63 (84.72)
  active                          84.61 (84.69)
  restaurants                     84.58 (84.67)
  food                            84.51 (84.58)
  nightlife                       83.45 (83.59)
  localflavor                    

## Tests

In [ ]:
if False:
    category_name = "Food"

    from data_prep import load_cached_root_category_businesses_df
    from data_prep import filter_root_category_businesses_uniq
    from data_prep import get_reviews_for_category

    dn_yelp_cached = data_yelp_path / "cached"
    df_food = load_cached_root_category_businesses_df(dn_yelp_cached, category_name, map_categories)
    # df_food = get_reviews_for_category(df, category_name, inv_cat_bids, map_categories, map_cat_name2id)

    df_uniq_food = filter_root_category_businesses_uniq(dn_yelp_cached, category_name, inv_cat_bids, map_categories, map_cat_name2id)

    print(f"reviews:")
    print(f"         all: {len(df):8d}")
    print(f"        food: {len(df_food):8d}")
    print(f" food-unique: {len(df_uniq_food):8d}")